<a href="https://www.kaggle.com/code/eliyahusanti/ps4e7-automl-fedot-catboost?scriptVersionId=186289021" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install fedot fedot[extra]
!pip uninstall xarray -y

In [ ]:
import pandas as pd
from fedot.api.main import Fedot
from fedot.core.pipelines.pipeline_builder import PipelineBuilder

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')
sub = pd.read_csv("/kaggle/input/playground-series-s4e7/sample_submission.csv")

In [ ]:
train.drop(columns=["id"], inplace=True)
test.drop(columns=["id"], inplace=True)

In [ ]:
def preprocess(df):
    df["Vehicle_Age"] = df["Vehicle_Age"].map({"< 1 Year": 0, "1-2 Year": 1, "> 2 Years": 2})
    df["Vehicle_Damage"] = df["Vehicle_Damage"].map({"No": 0, "Yes": 1})
    df["Gender"] = df["Gender"].map({"Male": 0, "Female": 1})
    return df

In [ ]:
train, test = [preprocess(df) for df in [train, test]]

In [ ]:
auto_model = Fedot(
    problem="classification",
    metric=["roc_auc"],
    preset="best_quality",
    with_tuning=True,
    timeout=5,
    cv_folds=10,
    seed=42,
    n_jobs=1,
    logging_level=10,
    initial_assumption=PipelineBuilder()
    .add_node(
        "scaling",
    )
    .add_node(
        "catboost",
        params={"use_eval_set": True, "use_best_model": True, "iterations": 10000, "n_jobs": -1},
    )
    .build(),
    use_pipelines_cache=False,
    use_auto_preprocessing=False,
)

In [ ]:
auto_model.fit(features=train, target="Response")

In [ ]:
prediction = auto_model.predict_proba(features=test)

In [ ]:
sub.Response = prediction.ravel()
sub.to_csv("submission.csv", index=False)

In [ ]:
print(auto_model.return_report().head(10))

In [ ]:
auto_model.current_pipeline.show()

In [ ]:
auto_model.current_pipeline.save(path="/kaggle/working/saved_pipelines", create_subdir=True, is_datetime_in_path=True)